## TEST CASES...

In [1]:
def test():
    b = [["B", "B", "B"],
         ["B", "B", "B"],
         ["B", "B", "B"]]
    
    test = get3x3(0,0,b)
    assert len(test) == 4, "4 values expected"
    assert test.count("B") == 4, "Incorrect number of bombs"
    
    test2 = get3x3(1,1,b)
    assert len(test2) == 9, "9 values expected"
    assert test2.count("B") == 9, "Incorrect number of bombs"
    
    test3 = get3x3(2,1,b)
    assert len(test3) == 6, "6 values expected"
    assert test3.count("B") == 6, "Incorrect number of bombs"
    
    b2 = [["B"],
          ["-"],
          ["B"]]
    
    test4 = get3x3(0,0,b2)
    assert len(test4) == 2, "2 values expected"
    assert test4.count("B") == 1, "Incorrect number of bombs"
    
    test5 = get3x3(1,0,b2)
    assert len(test5) == 3, "3 values expected"
    assert test5.count("B") == 2, "Incorrect number of bombs"
    
    b3 = [["B","-","-","-","B","-"]]
    
    test6 = get3x3(0,0,b3)
    assert len(test6) == 2, "2 values expected"
    assert test6.count("B") == 1, "Incorrect number of bombs"
    
    test7 = get3x3(0,2,b3)
    assert len(test7) == 3, "2 values expected"
    assert test7.count("B") == 0, "Incorrect number of bombs"
    
    test8 = get3x3(0,4,b3)
    assert len(test8) == 3, "3 values expected"
    assert test8.count("B") == 1, "Incorrect number of bombs"
    
    b4 = [["B"]]
    
    test9 = get3x3(0,0,b4)
    assert len(test9) == 1, "1 value expected"
    
    print("Testing Complete")

## Possible Solution

In [7]:
def get_square(x, y, board):
    """This function takes a board and returns the value at that square(x,y).
    """
    return board[x][y]


def get3x3(row, col, board):
    offsets = [(-1,-1), (0,-1), (+1,-1),
               (-1, 0), (0, 0), (+1, 0),
               (-1,+1), (0,+1), (+1,+1)]
    
    result = []
    for x, y in offsets:
        if row + x >=0 and col + y >= 0: # This check prevents 'wrapping' arround the board.
            try:
                result.extend(get_square(row + x, col + y, board))
            except IndexError:
                pass
    return result

# Testing...
test()

Testing Complete


Okay so let's explain this code a bit. 

The 'offsets' is a list of 9 values that is centered on the square (0,0). To reach the top-left square from (0,0) we would have to move one left(-1,0) and one up(0,-1). How we use these offsets is to take the real row, col values and add the offsets to row, col. If we do this for every offset we get the coridinates of every neighbour square *(please remember that adding a negetive number reduces the size of the number)*. Lets watch it in action:


In [11]:
offsets = [(-1,-1), (0,-1), (+1,-1), (-1, 0), (0, 0), (+1, 0), (-1,+1), (0,+1), (+1,+1)]

# row = 0, col = 0
print([(0 + x, 0 + y) for x, y in offsets])
# row = 10, col = 5
print([(10 + x, 5 + y) for x, y in offsets])
# row = 3, col = 3
print([(3 + x, 3 + y) for x, y in offsets])

[(-1, -1), (0, -1), (1, -1), (-1, 0), (0, 0), (1, 0), (-1, 1), (0, 1), (1, 1)]
[(9, 4), (10, 4), (11, 4), (9, 5), (10, 5), (11, 5), (9, 6), (10, 6), (11, 6)]
[(2, 2), (3, 2), (4, 2), (2, 3), (3, 3), (4, 3), (2, 4), (3, 4), (4, 4)]


Okay, but what if we are at the edges of the board? Couldn't we get IndexErrors if row/col + offsets push us of the board couldn't we also have problems when the square is(0, 0), how do we stop it wrapping arround and connecting the first square on the board(0,0) with the very last(-1,-1)?

The solution is to simple add a few checks:

    if row + x >=0 and col + y >= 0: 

This line of code handles the 'wrapping' situation by forcing all our indexes to be greater than or equal to zero. To fix the issue in the other direction we use try/except to ensure we stay on the board.

The final step is to get the value at that position on the board (I use the get_square function we defined earlier) and add it to our list of neibour squares. Once finished, we return said list.